In [1]:
#Import Packages 
import pyodbc
import pandas as pd
import numpy as np
import os, re, os.path
from datetime import datetime
from datetime import date
import shutil
from shutil import SameFileError
import ftplib
from ftplib import FTP #to connect to an FTP server
from crontab import CronTab
import csv
import subprocess
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Configuration
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
#%env
#%pwd

** 1.constantly looking against table with the recent date_processed time and run the process?  or recent row ID 

** 2.Do we have a main table to look up tables? 

** 3.need to add in destination source

In [3]:
# Connect to Server

# conn = pyodbc.connect('Driver={SQL Server};'
#                       'Server=USODCVSQL0256,7114;'
#                       'Database=MI_LEADS;'
#                       'Trusted_Connection=yes;')

# df = pd.read_sql_query('SELECT * FROM [dbo].[LEADS_LIST_FILE_CREATE_REQUEST]', conn)
# df.head(10)


conn = pyodbc.connect('Driver={SQL Server};'
                        'Server=USODCVSQL0256,7114;'
                        'Database=MI_LEADS;'
                        'UID=MISA;'
                        'PWD=Sra0!tmMz;'
                        'Trusted_Connection=no;')


df = pd.read_sql_query('SELECT * FROM [dbo].[LEADS_LIST_FILE_CREATE_REQUEST]', conn)
df.head(10)

,ROW_ID,TABLE_NAME,PROCESSED,DATETIME_PROCESSED,TACTIC
0,1,AB97883_DM_FINAL_TEST,1,2022-12-02 15:59:11.100,DM
1,2,1DMRRD_EN23X_10122022_B44,0,NaT,DM
2,3,1DRCYD_QW_WAD9_10062022_6C6,0,NaT,D2D
3,4,1EMSMC_EN00Y_10132022,1,2022-12-02 15:59:11.100,EM
4,5,1EMSMC_UJ00Y_10132022,0,NaT,EM
5,6,1TMVXI_NP24Y_10102022_145,0,NaT,TM
6,7,1DHINT_RG23XUT_LOCAL10112022_420,0,NaT,DH


# Create Files - Function 1 

In [4]:
#def checkTableExists_and_CreateFile(dbcon, tablename):
#    dbcur = dbcon.cursor()
#    print("Connected to SQL Server Successfully!",'\n\n')
        
#    dbcur.execute(
#        """
#        select count(*)
#        from information_schema.tables
#        where table_name = '{0}'
#        """.format(tablename.replace('\'', '\'\''))) #(select table_name from [dbo].[LEADS_LIST_FILE_CREATE_REQUEST] where datatime_processed = max(datatime_processed) )
    
#    if dbcur.fetchone()[0] == 1:
#        print('The table exists!','\n\n') 
        
#        # 1: create header label and export as text file 
#        table_name = tablename
#        sql = str("""select column_name 
#                     from information_schema.columns -- static table
#                     where table_name = '{0}';
#                  """).format(table_name) #can use subquery (select table_name from [dbo].[LEADS_LIST_FILE_CREATE_REQUEST])
#        #TEMPDB.INFORMATION_SCHEMA.COLUMNS
#        df = pd.read_sql(sql,con = dbcon)
#        df = df.T
#        df.columns = df.values[0]
#        df = df.reset_index(drop = True)
#        df = df.iloc[[0]]
#        df.drop([0]).to_csv('//pkdw0038l/dbmteam/Jenny_SH/txt_files_testing/data.txt', sep='|', header=True, index=False)
#        print(df.columns,'\n\n')

#        #cursor.execute("""
#        #select column_name
#        #from information_schema.columns
#        #where table_name = '{0}'
#        #""".format(tablename.replace('\'', '\'\'')))
#        
#        # to access first elements in a list of tuples
#        #for i in cursor.fetchall():
#            # turn <class 'str'> into <class 'pandas.core.series.Series'>
#            #print(i)
#        
#        
#        # 2: read in data and export as text file 
#        table_name = tablename
#        sql = str("""select * from {0};
#                  """).format(table_name)
#        df = pd.read_sql(sql,con = dbcon) 
#        df.to_csv('//pkdw0038l/dbmteam/Jenny_SH/txt_files_testing/data2.txt', sep='|', header=False, index=False)
#        #print(df)
#        
#        
#        # 3: combine output (header/data)
#        filenames = ['//pkdw0038l/dbmteam/Jenny_SH/txt_files_testing/data.txt', '//pkdw0038l/dbmteam/Jenny_SH/txt_files_testing/data2.txt']
#
#        try:
#            with open('//pkdw0038l/dbmteam/Jenny_SH/txt_files_testing/data3.txt', 'w') as outfile:
#                for names in filenames:
#                    # Open each file in read mode
#                    try:
#                        with open(names) as infile:
#                            # read the data from txt1 and txt2 and write it in txt3
#                            outfile.write(infile.read())
#                            outfile.write("\n")
#                    except:
#                        print("Read each file successfully",'\n\n')
#        except IOError:
#            print("Error raised with open files",'\n\n') 
#        else:
#            print ("File merge successfully!",'\n\n')
#    
#        # 4: load merge file into dataframe and Check the data structure
#        df = pd.read_csv('//pkdw0038l/dbmteam/Jenny_SH/txt_files_testing/data3.txt', sep='|')
#        print(df.head())
#        #dbcur.close()
#              
       
#        # 5: move files to destination folder (application process)
#        src_files = os.listdir(r"//pkdw0038l/dbmteam/Jenny_SH/txt_files_testing")
#        src = r"//pkdw0038l/dbmteam/Jenny_SH/txt_files_testing"
#        dest = r"Z:\\Jenny_testing" #r"C:\\Users\\AC27802\OneDrive - CenturyLink\Desktop\test_sh\copy_test_files" -- need to use mapped network drive 
#        
#        for file_name in src_files:
#            full_file_name = os.path.join(src, file_name)
#            if os.path.isfile(full_file_name):
#                shutil.copy(full_file_name, dest)
#                print("Source file copied to destination successfully :" , full_file_name,'\n\n')
#      
#    
#    else:
#        print('The table does not exist!')
#        return False

In [5]:
#checkTableExists_and_CreateFile(conn, "AB97883_DM_FINAL_TEST")

# Create Files - Function 2

In [6]:
# def checkTableExists_and_CreateFile_2(dbcon):
    
#     # connect to the server USODCVSQL0256,7114 at db MI_LEADS and see if there're tables in the file creation driver table
#     dbcur = dbcon.cursor()
#     print("Connected to SQL Server Successfully!",'\n\n')
        
#     dbcur.execute("""
#         select count(*)
#         from [dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
#         """) 
    
#     if dbcur.fetchone()[0] > 0:
#         print('There are files/tables in the driver table!','\n\n') 
        
#          # populate file creation driver table
#         sql = str("""select * 
#                      from [dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
#                      where PROCESSED = 0 and DATETIME_PROCESSED is null; -- static table
#                   """)
#         df = pd.DataFrame(pd.read_sql(sql,con = dbcon))
#         print(df['TACTIC'].tolist(),'\n\n')
        
#         # iterate rows in dataframe to identify different tactics and export files in different format
#         for tactic in df['TACTIC'].tolist(): #i, row in df.iterrows(): 
#             if tactic == 'DM ': 
#                 dm_sql = str("""select distinct TABLE_NAME
#                                 from [dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
#                                 where PROCESSED = 0 and DATETIME_PROCESSED is null
#                                 and TACTIC = 'DM'; -- static table
#                              """)
#                 dm_table_name = pd.DataFrame(pd.read_sql(dm_sql, con = dbcon))
#                 #print(dm_table_name['TABLE_NAME'].tolist(),'\n\n')
         
#                # read in table and output files to source folders
#                 for dm_tablename in dm_table_name['TABLE_NAME'].tolist():
#                     read_dm_table_sql = str("""select * from [{0}];
#                                             """).format(dm_tablename)
#                     df_each_dm_table = pd.read_sql(read_dm_table_sql, con = dbcon) 
#                     df_each_dm_table.to_csv('//pkdw0038l/dbmteam/Jenny_SH/txt_files_testing/' + dm_tablename + date.today().strftime("%m%d%Y")+'.txt',  sep='|', header=True, index=False)
                    
#                 # update table records with processed = 1 and system datetime stamp 
#                     update_dm_table_sql = dbcur.execute ("UPDATE LEADS_LIST_FILE_CREATE_REQUEST SET PROCESSED = 1, DATETIME_PROCESSED = sysdatetime() where TABLE_NAME = ?", (dm_tablename))
#                     dbcon.commit()
    
#             if tactic == 'TM ':  
#                 tm_sql = str("""select distinct TABLE_NAME
#                                 from [dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
#                                 where PROCESSED = 0 and DATETIME_PROCESSED is null
#                                 and TACTIC = 'TM'; -- static table
#                              """)
                
#                 tm_table_name = pd.DataFrame(pd.read_sql(tm_sql, con = dbcon))
#                 #print(tm_table_name['TABLE_NAME'].tolist(),'\n\n')
                
#                 for tm_tablename in tm_table_name['TABLE_NAME'].tolist():
#                     read_tm_table_sql = str("""select * from [{0}];
#                                             """).format(tm_tablename)
#                     df_each_tm_table = pd.read_sql(read_tm_table_sql, con = dbcon) 
#                     df_each_tm_table.to_csv('//pkdw0038l/dbmteam/Jenny_SH/txt_files_testing/' + tm_tablename + date.today().strftime("%m%d%Y")+'.txt',  sep='?', header=True, index=False)
                    
#                 # update table records with processed = 1 and system datetime stamp 
#                     update_tm_table_sql = dbcur.execute ("UPDATE LEADS_LIST_FILE_CREATE_REQUEST SET PROCESSED = 1, DATETIME_PROCESSED = sysdatetime() where TABLE_NAME = ?", (tm_tablename))
#                     dbcon.commit()   

                
#             if tactic == 'EM ':   
#                 em_sql = str("""select distinct TABLE_NAME
#                                 from [dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
#                                 where PROCESSED = 0 and DATETIME_PROCESSED is null
#                                 and TACTIC = 'EM'; -- static table
#                               """)
                
#                 em_table_name = pd.DataFrame(pd.read_sql(em_sql, con = dbcon))
#                 #print(em_table_name['TABLE_NAME'].tolist(),'\n\n')
                
#                 for em_tablename in em_table_name['TABLE_NAME'].tolist():
#                     read_em_table_sql = str("""select * from [{0}];
#                                             """).format(em_tablename)
#                     df_each_em_table = pd.read_sql(read_em_table_sql, con = dbcon) 
#                     df_each_em_table.to_csv('//pkdw0038l/dbmteam/Jenny_SH/txt_files_testing/' + em_tablename + date.today().strftime("%m%d%Y")+'.csv',  sep=',', header=True, index=False)
               
#                 # update table records with processed = 1 and system datetime stamp 
#                     update_em_table_sql = dbcur.execute ("UPDATE LEADS_LIST_FILE_CREATE_REQUEST SET PROCESSED = 1, DATETIME_PROCESSED = sysdatetime() where TABLE_NAME = ?", (em_tablename))
#                     dbcon.commit()   

    

#             if tactic == 'D2D':    
#                 D2D_sql = str("""select distinct TABLE_NAME
#                                 from [dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
#                                 where PROCESSED = 0 and DATETIME_PROCESSED is null
#                                 and TACTIC = 'D2D'; -- static table
#                               """)
                
#                 D2D_table_name = pd.DataFrame(pd.read_sql(D2D_sql, con = dbcon))
#                 #print(D2D_table_name['TABLE_NAME'].tolist(),'\n\n')
                
#                 for D2D_tablename in D2D_table_name['TABLE_NAME'].tolist():
#                     read_d2d_table_sql = str("""select * from [{0}];
#                                             """).format(D2D_tablename)
#                     df_each_d2d_table = pd.read_sql(read_d2d_table_sql, con = dbcon) 
#                     df_each_d2d_table.to_csv('//pkdw0038l/dbmteam/Jenny_SH/txt_files_testing/' + D2D_tablename + date.today().strftime("%m%d%Y")+'.csv',  sep=',', header=True, index=False)
                
#                 # update table records with processed = 1 and system datetime stamp 
#                     update_d2d_table_sql = dbcur.execute ("UPDATE LEADS_LIST_FILE_CREATE_REQUEST SET PROCESSED = 1, DATETIME_PROCESSED = sysdatetime() where TABLE_NAME = ?", (D2D_tablename))
#                     dbcon.commit()   
         
            
#             if tactic == 'DH ':    
#                 DH_sql = str("""select distinct TABLE_NAME
#                                 from [dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
#                                 where PROCESSED = 0 and DATETIME_PROCESSED is null
#                                 and TACTIC = 'DH'; -- static table
#                               """)
                
#                 DH_table_name = pd.DataFrame(pd.read_sql(DH_sql, con = dbcon))
#                 #print(DH_table_name['TABLE_NAME'].tolist(),'\n\n')
                
#                 for DH_tablename in DH_table_name['TABLE_NAME'].tolist():
#                     read_dh_table_sql = str("""select * from [{0}];
#                                             """).format(DH_tablename)
#                     df_each_dh_table = pd.read_sql(read_dh_table_sql, con = dbcon) 
#                     df_each_dh_table.to_csv('//pkdw0038l/dbmteam/Jenny_SH/txt_files_testing/' + DH_tablename + date.today().strftime("%m%d%Y")+'.csv',  sep=',', header=True, index=False)

#                 # update table records with processed = 1 and system datetime stamp 
#                     update_dh_table_sql = dbcur.execute ("UPDATE LEADS_LIST_FILE_CREATE_REQUEST SET PROCESSED = 1, DATETIME_PROCESSED = sysdatetime() where TABLE_NAME = ?", (DH_tablename))
#                     dbcon.commit()   
    
#     else:
#         print('There are no files/tables in the driver table!')
#        # return False
    
    
#     # move files to destination folder #\CCM_AUTOMATION\Adhoc\Output
#     src_files = os.listdir(r"//pkdw0038l/dbmteam/Jenny_SH/txt_files_testing")
#     src = r"//pkdw0038l/dbmteam/Jenny_SH/txt_files_testing"
#     dest = r"Y:\CCM_AUTOMATION\Adhoc\Output\Testing_Files_JH" #r"C:\\Users\\AC27802\OneDrive - CenturyLink\Desktop\test_sh\copy_test_files" -- need to use mapped network drive 
     
#     try:
#         for file_name in src_files:
#             dest_full_file_name = os.path.join(dest, file_name)
#             src_full_file_name = os.path.join(src, file_name)
#             if os.path.isfile(src_full_file_name):
#                 shutil.copy(src_full_file_name, dest)
#                 print("Source", src_full_file_name,'\n\n'
#                       "Destination",dest_full_file_name,'\n\n'
#                       "copied to destination successfully",'\n\n') 

#     except IOError:
#         print("Error raised with moving files to destination!",'\n\n') 
#     else:
#         print("Source file copied to destination successfully!",'\n\n')

#    # dbcon.close() # close server connection 

In [7]:
# checkTableExists_and_CreateFile_2(conn)

# Create Files - Function 3

In [8]:
def checkTableExists_and_CreateFile_3(dbcon):
    
    # connect to the server USODCVSQL0256,7114 at db MI_LEADS and see if there're tables in the file creation driver table
    dbcur = dbcon.cursor()
    print("Connected to SQL Server Successfully!",'\n\n')
        
    dbcur.execute("""
                  select count(*)
                  from [dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
                  """) 
    
    if dbcur.fetchone()[0] > 0:
        print('There are files/tables in the driver table!','\n\n') 
        
        # Read table and check if column contains processed = 1 and DATETIME_PROCESSED, if contains 1 then ignore and only process processed = 0 
        # populate file creation driver table
        sql = str("""select * 
                     from [dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
                      --where PROCESSED = 0 and DATETIME_PROCESSED is null
                      ; -- static table
                  """)
                
        df = pd.DataFrame(pd.read_sql(sql,con = dbcon))
        print(df['TACTIC'].tolist(),'\n\n')
        
        
        
        for ind, row in df.iterrows():  #print(row['PROCESSED'], row['TACTIC'], row['TABLE_NAME'])
            if (row['PROCESSED'] == 0):
                if (row['TACTIC'].strip() == 'DM'):
                    
                    # iterate rows in dataframe to identify different tactics and export files in different format
                    dm_sql = str("""select distinct TABLE_NAME
                                    from [dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
                                    where PROCESSED = 0 
                                    and DATETIME_PROCESSED is null
                                    and TACTIC = 'DM'
                                    and TABLE_NAME = '{0}'
                                    ; -- static table
                                 """).format(row['TABLE_NAME'])
                    dm_table_name = pd.DataFrame(pd.read_sql(dm_sql, con = dbcon))
                    print(dm_table_name['TABLE_NAME'].tolist(),'\n\n')
                    
                    # read in table and output files to source folders
                    for dm_tablename in dm_table_name['TABLE_NAME'].tolist():
                        read_dm_table_sql = str("""select * from [{0}];
                                                """).format(dm_tablename)
                        df_each_dm_table = pd.read_sql(read_dm_table_sql, con = dbcon) 
                        df_each_dm_table.to_csv('//pkdw0038l/dbmteam/Jenny_SH/txt_files_testing/' + dm_tablename + date.today().strftime("%m%d%Y")+'.txt',  sep='|', header=True, index=False)
                    
                        # update table records with processed = 1 and system datetime stamp 
                        update_dm_table_sql = dbcur.execute ("UPDATE LEADS_LIST_FILE_CREATE_REQUEST SET PROCESSED = 1, DATETIME_PROCESSED = sysdatetime() where TABLE_NAME = ?", (dm_tablename))
                        dbcon.commit()
                        
                if (row['TACTIC'].strip() == 'TM'):
                    tm_sql = str("""select distinct TABLE_NAME
                                    from [dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
                                    where PROCESSED = 0 
                                    and DATETIME_PROCESSED is null
                                    and TACTIC = 'TM'
                                    and TABLE_NAME = '{0}'
                                    ; -- static table
                                 """).format(row['TABLE_NAME'])       
                    tm_table_name = pd.DataFrame(pd.read_sql(tm_sql, con = dbcon))
                    print(tm_table_name['TABLE_NAME'].tolist(),'\n\n')
                
                    for tm_tablename in tm_table_name['TABLE_NAME'].tolist():
                        read_tm_table_sql = str("""select * from [{0}];
                                                """).format(tm_tablename)
                        df_each_tm_table = pd.read_sql(read_tm_table_sql, con = dbcon) 
                        df_each_tm_table.to_csv('//pkdw0038l/dbmteam/Jenny_SH/txt_files_testing/' + tm_tablename + date.today().strftime("%m%d%Y")+'.txt',  sep='?', header=True, index=False)
                    
                        # update table records with processed = 1 and system datetime stamp 
                        update_tm_table_sql = dbcur.execute ("UPDATE LEADS_LIST_FILE_CREATE_REQUEST SET PROCESSED = 1, DATETIME_PROCESSED = sysdatetime() where TABLE_NAME = ?", (tm_tablename))
                        dbcon.commit()   

                if (row['TACTIC'].strip() == 'EM'):
                    em_sql = str("""select distinct TABLE_NAME
                                    from [dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
                                    where PROCESSED = 0 
                                    and DATETIME_PROCESSED is null
                                    and TACTIC = 'EM'
                                    and TABLE_NAME = '{0}'                                    
                                    ; -- static table
                                  """).format(row['TABLE_NAME'])
                    em_table_name = pd.DataFrame(pd.read_sql(em_sql, con = dbcon))
                    print(em_table_name['TABLE_NAME'].tolist(),'\n\n')
                
                    for em_tablename in em_table_name['TABLE_NAME'].tolist():
                        read_em_table_sql = str("""select * from [{0}];
                                                """).format(em_tablename)
                        df_each_em_table = pd.read_sql(read_em_table_sql, con = dbcon) 
                        df_each_em_table.to_csv('//pkdw0038l/dbmteam/Jenny_SH/txt_files_testing/' + em_tablename + date.today().strftime("%m%d%Y")+'.csv',  sep=',', header=True, index=False)
               
                        # update table records with processed = 1 and system datetime stamp 
                        update_em_table_sql = dbcur.execute ("UPDATE LEADS_LIST_FILE_CREATE_REQUEST SET PROCESSED = 1, DATETIME_PROCESSED = sysdatetime() where TABLE_NAME = ?", (em_tablename))
                        dbcon.commit()           

                        
                if (row['TACTIC'].strip() == 'D2D'):
                    D2D_sql = str("""select distinct TABLE_NAME
                                      from [dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
                                      where PROCESSED = 0 
                                      and DATETIME_PROCESSED is null
                                      and TACTIC = 'D2D'
                                      and TABLE_NAME = '{0}'   
                                      ; -- static table
                                  """).format(row['TABLE_NAME'])
                    D2D_table_name = pd.DataFrame(pd.read_sql(D2D_sql, con = dbcon))
                    print(D2D_table_name['TABLE_NAME'].tolist(),'\n\n')
                
                    for D2D_tablename in D2D_table_name['TABLE_NAME'].tolist():
                        read_d2d_table_sql = str("""select * from [{0}];
                                                 """).format(D2D_tablename)
                        df_each_d2d_table = pd.read_sql(read_d2d_table_sql, con = dbcon) 
                        df_each_d2d_table.to_csv('//pkdw0038l/dbmteam/Jenny_SH/txt_files_testing/' + D2D_tablename + date.today().strftime("%m%d%Y")+'.csv',  sep=',', header=True, index=False)
                
                        # update table records with processed = 1 and system datetime stamp 
                        update_d2d_table_sql = dbcur.execute ("UPDATE LEADS_LIST_FILE_CREATE_REQUEST SET PROCESSED = 1, DATETIME_PROCESSED = sysdatetime() where TABLE_NAME = ?", (D2D_tablename))
                        dbcon.commit()   
       
        
                if (row['TACTIC'].strip() == 'DH'):
                    DH_sql = str("""select distinct TABLE_NAME
                                     from [dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
                                     where PROCESSED = 0 
                                     and DATETIME_PROCESSED is null
                                     and TACTIC = 'DH'
                                     and TABLE_NAME = '{0}'  
                                     ; -- static table
                                   """).format(row['TABLE_NAME'])
                    DH_table_name = pd.DataFrame(pd.read_sql(DH_sql, con = dbcon))
                    print(DH_table_name['TABLE_NAME'].tolist(),'\n\n')
                
                    for DH_tablename in DH_table_name['TABLE_NAME'].tolist():
                        read_dh_table_sql = str("""select * from [{0}];
                                                """).format(DH_tablename)
                            
                        df_each_dh_table = pd.read_sql(read_dh_table_sql, con = dbcon) 
                        df_each_dh_table.to_csv('//pkdw0038l/dbmteam/Jenny_SH/txt_files_testing/' + DH_tablename + date.today().strftime("%m%d%Y")+'.csv',  sep=',', header=True, index=False)

                        # update table records with processed = 1 and system datetime stamp 
                        update_dh_table_sql = dbcur.execute ("UPDATE LEADS_LIST_FILE_CREATE_REQUEST SET PROCESSED = 1, DATETIME_PROCESSED = sysdatetime() where TABLE_NAME = ?", (DH_tablename))
                        dbcon.commit()           
       
    
            
            elif row['PROCESSED'] == 1:
                print('Table have already been processed, please check back later!')
                
            else:
                print('Table have already been processed, please check back later!')


    
    else:
        print('There are no files/tables in the driver table!')
       # return False
    
    
    # move files to destination folder #\CCM_AUTOMATION\Adhoc\Output
    src_files = os.listdir(r"//pkdw0038l/dbmteam/Jenny_SH/txt_files_testing")
    src = r"//pkdw0038l/dbmteam/Jenny_SH/txt_files_testing"
    dest = r"V:\CCM_AUTOMATION\Adhoc\Output\Testing_Files_JH" #r"C:\\Users\\AC27802\OneDrive - CenturyLink\Desktop\test_sh\copy_test_files" -- need to use mapped network drive 
     
    try:
        for file_name in src_files:
            dest_full_file_name = os.path.join(dest, file_name)
            src_full_file_name = os.path.join(src, file_name)
            if os.path.isfile(src_full_file_name):
                shutil.copy(src_full_file_name, dest)
                print("Source", src_full_file_name,'\n\n'
                      "Destination",dest_full_file_name,'\n\n'
                      "copied to destination successfully",'\n\n') 

    except IOError:
        print("Error raised with moving files to destination!",'\n\n') 
    else:
        print("Source file copied to destination successfully!",'\n\n')

   # dbcon.close() # close server connection 

In [9]:
checkTableExists_and_CreateFile_3(conn)

Connected to SQL Server Successfully! 


There are files/tables in the driver table! 


['DM ', 'DM ', 'D2D', 'EM ', 'EM ', 'TM ', 'DH '] 


Table have already been processed, please check back later!
['1DMRRD_EN23X_10122022_B44'] 


['1DRCYD_QW_WAD9_10062022_6C6'] 


Table have already been processed, please check back later!
['1EMSMC_UJ00Y_10132022'] 


['1TMVXI_NP24Y_10102022_145'] 


['1DHINT_RG23XUT_LOCAL10112022_420'] 


Source //pkdw0038l/dbmteam/Jenny_SH/txt_files_testing\1DHINT_RG23XUT_LOCAL10112022_42012022022.csv 

Destination Y:\CCM_AUTOMATION\Adhoc\Output\Testing_Files_JH\1DHINT_RG23XUT_LOCAL10112022_42012022022.csv 

copied to destination successfully 


Source //pkdw0038l/dbmteam/Jenny_SH/txt_files_testing\1DMRRD_EN23X_10122022_B4412022022.txt 

Destination Y:\CCM_AUTOMATION\Adhoc\Output\Testing_Files_JH\1DMRRD_EN23X_10122022_B4412022022.txt 

copied to destination successfully 


Source //pkdw0038l/dbmteam/Jenny_SH/txt_files_testing\1DRCYD_QW_WAD9_10062022_6C612022022.c

In [10]:
df = pd.read_sql_query('SELECT * FROM [dbo].[LEADS_LIST_FILE_CREATE_REQUEST]', conn)
df.head(10)

,ROW_ID,TABLE_NAME,PROCESSED,DATETIME_PROCESSED,TACTIC
0,1,AB97883_DM_FINAL_TEST,1,2022-12-02 15:59:11.100,DM
1,2,1DMRRD_EN23X_10122022_B44,1,2022-12-02 16:00:33.117,DM
2,3,1DRCYD_QW_WAD9_10062022_6C6,1,2022-12-02 16:01:43.447,D2D
3,4,1EMSMC_EN00Y_10132022,1,2022-12-02 15:59:11.100,EM
4,5,1EMSMC_UJ00Y_10132022,1,2022-12-02 16:01:45.447,EM
5,6,1TMVXI_NP24Y_10102022_145,1,2022-12-02 16:01:45.777,TM
6,7,1DHINT_RG23XUT_LOCAL10112022_420,1,2022-12-02 16:01:47.527,DH
